In [1]:
import pandas as pd
df = pd.read_csv('/content/train-3.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [3]:
df = df.fillna('Not Available')

In [4]:
df['news'] = df['title'] +'\n '+ df['text']

In [5]:
print(df['news'][0])

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It
 House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) 
With apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. 
As we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Inte

In [6]:
from nltk.corpus import stopwords

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
sw = stopwords.words("english")
len(sw)

179

In [9]:
import re
lines = []
for i in str(df.title):
    words = ""
    Q = i.lower()
    Q = re.sub("[^a-z ]","",Q)
    Q = Q.split(" ")
    #print(Q)
    for j in Q:      
        if(j not in sw):
            words = words + " " + j
    lines.append(words)

In [10]:
lines[0]

' '

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
ti = TfidfVectorizer()
X = ti.fit_transform(pd.Series(lines)).toarray()
Y = df.label

ValueError: ignored

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.2,random_state=21,stratify=Y)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10)
nn = Sequential()
nn.add(Dense(512,input_dim=22870,activation='relu'))
nn.add(Dropout(0.3))
nn.add(Dense(512,activation='relu'))
nn.add(Dropout(0.3))
nn.add(Dense(512,activation='relu'))
nn.add(Dropout(0.3))
nn.add(Dense(1,activation='sigmoid'))
nn.compile(loss='binary_crossentropy',metrics='accuracy',optimizer='adam')
model = nn.fit(xtrain,ytrain, validation_data=(xtest,ytest),callbacks=es,epochs=100)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.history["accuracy"])
plt.plot(model.history["val_accuracy"])

In [ ]:
plt.plot(model.history["loss"])
plt.plot(model.history["val_loss"])

In [ ]:
tr_pred = nn.predict(xtrain)
ts_pred = nn.predict(xtest)

In [ ]:
tr_pred1 = []
for i in tr_pred:
    if i > 0.5:
        tr_pred1.append(1)
    else:
        tr_pred1.append(0)

In [ ]:
ts_pred1 = []
for i in ts_pred:
    if i > 0.5:
        ts_pred1.append(1)
    else:
        ts_pred1.append(0)

In [ ]:
from sklearn.metrics import accuracy_score
tr_acc = accuracy_score(ytrain,tr_pred1)
ts_acc = accuracy_score(ytest,ts_pred1)
print(tr_acc,ts_acc)

In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(ytest,ts_pred1)
import seaborn as sb
sb.heatmap(cf,annot=True)

In [ ]:
from sklearn.metrics import classification_report
cr = classification_report(ytest,ts_pred1)
print(cr)

In [ ]:
df_test = pd.read_csv('/kaggle/input/fake-news/test.csv')

In [ ]:
df_test.isna().sum()

In [ ]:
df_test = df_test.fillna('Not Available')

In [ ]:
import re
lines_test = []
for i in df_test.title:
    words = ""
    Q = i.lower()
    Q = re.sub("[^a-z ]","",Q)
    Q = Q.split(" ")
    #print(Q)
    for j in Q:      
        if(j not in sw):
            words = words + " " + j
    lines_test.append(words)

In [ ]:
X_test = ti.transform(pd.Series(lines_test)).toarray()

In [ ]:
X_test

In [ ]:
test_pred = nn.predict(X_test)

In [ ]:
test_pred1 = []
for i in test_pred:
    if i>0.5:
        test_pred1.append(1)
    else:
        test_pred1.append(0)

In [ ]:
idx = list(df_test['id'])
df_sub = pd.DataFrame({'id':idx, 'label':test_pred1})

In [ ]:
df_sub.head()

In [ ]:
df_sub.groupby(by='label').count()

In [ ]:
df_sub.groupby(by='label').count().plot(kind='bar')

In [ ]:
df_sub.to_csv('submission.csv',index=False)